In [ ]:
from sage.all import log, randint
import numpy as np
from ckks_in_sagemath.ckks_package.ckks import CKKS
from ckks_in_sagemath.ckks_package.poly import Poly
from ckks_in_sagemath.ckks_package.fast_dft import get_grouped_F
from ckks_in_sagemath.ckks_package.bit_rev import bit_rev

In [2]:
def ext_bit_rev(i, B, log_B=None):
    if log_B is None:
        log_B = log(B, 2)
    q, r = divmod(i, B)
    return q * B + bit_rev(r, log_B)


def ext_bit_rev_vector(v, B):
    n = len(v)
    log_B = log(B, 2)
    return np.array([v[ext_bit_rev(i, B, log_B)] for i in range(n)])

In [3]:
# Basic parameters

N = 2**15
q = 2**29
p = 2**22
delta = 2**32

n = 2**7  # Number of coefficients
h = 2**6
log_radix = 4
L = ceil(log(n // 2, 2) / log_radix) + log(h, 2) + 2

# Secret key generation

B = N // h
sk_coeffs = np.zeros(N, dtype=int)
sk_coeffs[0] = 1
for b_coeffs in range(1, h):
    j = randint(0, B - 1)
    sk_coeffs[b_coeffs * B + j] = 1
sk = Poly(sk_coeffs, N)

# CKKS configuration

CKKS.config(N, N // 2, L, q, p, delta, print_messages=True)
CKKS.key_gen(sk=sk)

The CKKS configuration is done!


In [4]:
# swk generation

for i in range(0, log(N, 2)):
    # Rotation by powers of two
    k = 5 ** (2**i)
    swk = CKKS.get_galois_swk(k, sk)
swk = CKKS.get_galois_swk(-1, sk)  # Conjugation

# Bootstrapping keys

s_tilde_list = []
cs_list = []
for u in range(2 * n):
    s_tilde = np.zeros(N // 2, dtype=np.complex128)
    for k in range(B // (2 * n)):
        for b in range(h):
            for a in range(n):
                s_tilde[k * h * n + b * n + a] = sk_coeffs[
                    b * B + u * B // (2 * n) + k
                ]
    s_tilde = ext_bit_rev_vector(s_tilde, n // 2)
    s_tilde_list.append(s_tilde)
    cs_list.append(
        CKKS.enc_poly_with_sk(
            CKKS.encode(s_tilde, is_boot=True), sk, is_boot=True
        )
    )

# Configure SlotToCoeff

eta = np.zeros(n, np.complex128)
for i in range(n):
    eta[i] = 1 if i < n // 2 else 1j
enc_eta = CKKS.encode(eta, is_boot=True)

grouped_F = get_grouped_F(n // 2, log_radix, False).copy()

grouped_poly_F = [CKKS.get_poly_matrix(A, is_boot=True) for A in grouped_F]

rotation_indices = []
for poly_matrix in grouped_poly_F:
    rotation_indices += CKKS.get_BSGS_rotation_indices(poly_matrix)
for k in rotation_indices:
    CKKS.get_galois_swk(5**k, sk)

In [5]:
# Ciphertext creation

m = Poly([randint(-p, p) if i % (N // n) == 0 else 0 for i in range(N)], N)
ct = CKKS.enc_poly_with_pk(m, CKKS.pk) % q

In [6]:
# Decryption matrix

C = np.zeros((N, n), dtype=int)
a_coeffs, b_coeffs = ct.a.coeffs, ct.b.coeffs
for j in range(n):
    for i in range(N):
        if N // n * j >= i:
            C[i, j] = a_coeffs[N // n * j - i]
        else:
            C[i, j] = -a_coeffs[N + N // n * j - i]
    C[0, j] += int(b_coeffs[N // n * j])

# Coefficient encodings

e_list = []
E_list = []
delta_factor = (q / (4 * delta * np.pi)) ** (1 / h)
exp_factor = 2 * np.pi * 1j / q
for u in range(2 * n):
    e = np.zeros(N // 2, dtype=np.complex128)
    for k in range(B // (2 * n)):
        for b in range(h):
            for a in range(n):
                e[k * h * n + b * n + a] = C[b * B + u * B // (2 * n) + k, a]
    e = np.exp(e * exp_factor) * delta_factor
    e = ext_bit_rev_vector(e, n // 2)
    e_list.append(e)
    E_list.append(CKKS.encode(e, is_boot=True))

# Bootstrapping without SlotToCoeff

out = CKKS.enc_poly_without_error(0, is_boot=True)
for u in range(2 * n):
    out += cs_list[u] @ E_list[u]
out = CKKS.trace(out, N // 2, h * n)
out = CKKS.product(out, h * n, n)
out = out - out.conjugate()
out = -(1j * out)

# SlotToCoeff

out = enc_eta @ out
out = out.trace(n, n // 2)

for A in grouped_poly_F:
    out = out.BSGS_left_mult(A)

out = out.boot_to_nonboot()

In [7]:
print(out.dec_to_poly(sk))
print(m)
print(out.l)
print(out.get_precision(ct, sk, n // 2))

553583 + 30X + 8X^2 - 88X^3 - 8X^4 + 45X^5 + 57X^6 + 37X^7 - 9X^8 + 3X^9 + 26X^10 + 29X^11 + 24X^12 + 62X^13 + 57X^14 - 59X^15 - 14X^16 - 55X^17 - 24X^18 - 74X^19 + 43X^20 - 4X^21 - 90X^22 + 26X^23 - 14X^24 - 48X^25 + 39X^26 - 66X^27 - 3X^28 - 72X^29 - 23X^30 - 41X^31 + 70X^32 - 95X^33 - 81X^34 + 45X^35 - 62X^36 + 22X^37 - 38X^38 + 72X^39 + 24X^40 - 62X^41 + 36X^42 - 55X^43 + 4X^44 - 177X^45 + 37X^46 + 38X^47 + 51X^48 - 62X^49 + X^50 + 149X^51 - 36X^52 + 74X^53 + 74X^54 + 3X^55 + 19X^56 - 55X^57 + 45X^58 + 18X^59 - 77X^60 - 38X^61 + 111X^62 + 111X^63 - 53X^64 - 40X^65 + 78X^66 - 28X^67 + 58X^68 + 75X^69 + 19X^70 + 17X^71 - 83X^72 + 25X^73 + 96X^74 + 39X^75 + 54X^76 - 61X^77 + 29X^78 + 37X^79 + 88X^80 + 16X^81 + 74X^82 + 24X^83 - 38X^84 - 80X^85 + 115X^86 - 30X^87 - 53X^88 - 51X^89 + 56X^90 + 28X^91 + X^92 - 30X^93 - 57X^94 + 40X^95 + 36X^96 - 61X^97 - 73X^98 + 19X^99 + 10X^100 - 9X^101 - 14X^102 - 94X^103 + 18X^105 - 14X^106 - 101X^107 - 83X^108 - 29X^109 + 20X^110 + 14X^111 - 28X^112 